# 24-1차 AI 기초 연수과정 Linear Classifier

**필요한 라이브러리, 모듈 가져오기**




In [11]:
from tqdm import tqdm
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [3]:
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,            ## 50,000 / 4 = 12,500
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,              ## 10,000 / 4 = 2,500
                                         shuffle=False, num_workers=2)

100%|██████████| 170498071/170498071 [00:01<00:00, 102136191.42it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [22]:
# 모델 정의
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(32 * 32 * 3, 10)

    def forward(self, x):
        x = x.view(-1, 32 * 32 * 3)
        x = F.relu(x)
        output = self.fc1(x)
        return output

In [24]:
# 모델 학습
net = Net()

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

for epoch in tqdm(range(1)):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # 입력 데이터
        inputs, labels = data

        # 모델 실행
        outputs = net(inputs)

        # 손실 계산
        loss = criterion(outputs, labels)

        # 모델 최적화
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 손실 기록
        loss = criterion(outputs, labels)
        # tqdm.write('Epoch: {} Batch: {} Loss: {:.3f}'.format(epoch + 1, i + 1, loss.item()))
        running_loss += loss.item()
        # if i % 2000 == 1999:    # 2000번마다 출력
        #     print('[%d, %5d] loss: %.3f' %
        #           (epoch + 1, i + 1, running_loss / 2000))
        #     running_loss = 0.

    tqdm.write(' Epoch: {} Average Loss: {:.3f}'.format(epoch + 1, running_loss / len(trainloader)))

100%|██████████| 1/1 [01:04<00:00, 64.57s/it]

 Epoch: 1 Average Loss: 6.080


In [20]:
# 모델 평가
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10000 test images: 27 %
